# Spel zonder Spelelement

Ontwikkeld met versies:
- Python 3.11.6
- Pandas 2.0.3
- ipywidgets 8.1.2

In [ ]:
# Imports
import pandas as pd
import random
import time
import ipywidgets as widgets

from IPython.display import display, Image, HTML, clear_output

In [ ]:
## CARD
class Card:
	def __init__(self, rank, suit):
		self.rank = rank
		self.suit = suit
		self.name = self._get_card_name(rank)
		if self.name in ["Jack", "Queen", "King"]:
			self.image_path = f"_cards/{self.name.lower()}_of_{suit.lower()}2.png"
		else:
			self.image_path = f"_cards/{self.name.lower()}_of_{suit.lower()}.png"
		 
	def __str__(self):
		return f"{self.name} of {self.suit}"

	def _get_card_name(self, rank):
		if rank == 11:
			return "Jack"
		elif rank == 12:
			return "Queen"
		elif rank == 13:
			return "King"
		elif rank == 14:
			return "Ace"
		else:
			return str(rank)
		
	def get_image_html(self):
		return f'<img src="{self.image_path}" alt="{self.name} of {self.suit}">'

## DECK
class Deck:
	def __init__(self, seed=42):
		self.cards = []
		for suit in ['Hearts', 'Diamonds', 'Clubs', 'Spades']:
			for rank in range(2, 15):
				self.cards.append(Card(rank, suit))
		random.seed(seed)
		self.shuffle()

	def shuffle(self):
		random.shuffle(self.cards)

	def deal(self, num_players):
		hands = [[] for _ in range(num_players)]
		for i, card in enumerate(self.cards):
			hands[i % num_players].append(card)
		return hands

# PLAYER
class Player:
	def __init__(self, name):
		self.name = name
		self.hand = []

	def play_card(self):
		return self.hand.pop(0)

	def add_cards(self, cards):
		self.hand.extend(cards)

	def has_cards(self):
		return len(self.hand) > 0

	def print_hand(self):
		print(f"{self.name}'s hand:")
		for card in self.hand:
			print(card)

# GAME
class Game:

	DEFAULT_NAMES = ['Mawkmawk', 'Rap', 'Irun', 'Kurwa', 'Gerbood', 'Comar']

	def __init__(self, seed=42, player_names=DEFAULT_NAMES):
		self.player_names = player_names
		self.num_players = len(self.player_names)
		self.deck = Deck(seed)
		self.players = [Player(name) for name in self.player_names]
		self.hands = self.deck.deal(self.num_players)
		self.deal_cards()
		self.played_cards = []
		self.winning_cards = []

	def deal_cards(self):
		for player, hand in zip(self.players, self.hands):
			player.add_cards(hand)
	
	def show_all_hands(self):
		max_hand_length = max(len(player.hand) for player in self.players)
		hand_data = {player.name: [card for card in player.hand] + [None] * (max_hand_length - len(player.hand)) for player in self.players}
		df = pd.DataFrame(hand_data).T
		return df

	def display_played_cards(self, played_cards, winners=None):
		html = '<div style="display: flex; justify-content: space-around;">'
		for player, card in played_cards:
			html += '<div style="text-align: center;">'
			if winners and player in winners:
				html += f'<div><b>{player.name}</b></div>'
			else:
				html += f'<div>{player.name}</div>'
			html += f'<img src="{card.image_path}" alt="{card}" style="width: 120px; height: auto;">'
			html += '</div>'
		html += '</div>'
		display(HTML(html))
		
	def play_round(self):
		self.played_cards = [(player, player.play_card()) for player in self.players if player.has_cards()]
		if not self.played_cards:
			print('No cards played')
			return None  # No cards left to play

		highest_rank = max(self.played_cards, key=lambda x: x[1].rank)[1].rank #  x[1] represents the Card object
		# Find the players with the highest rank cards
		winners = [player for player, card in self.played_cards if card.rank == highest_rank]

		# Display the played cards after determining the winners
		self.display_played_cards(self.played_cards, winners)

		# Build the winning card pile
		self.winning_cards.extend([card for player, card in self.played_cards])

		# Clear the played cards.
		self.played_cards = []

		if len(winners) > 1:
			# In case of a draw, we start a battle!
			print("It's a draw! Battle!!")			
			
			while len(winners) > 1:
				self.played_cards = [(player, player.play_card()) for player in winners if player.has_cards()]
				if not self.played_cards:
					print('No cards played in the battle')
					return None  # No cards left to play

				# Find the highest rank among the newly played cards
				highest_rank = max(self.played_cards, key=lambda x: x[1].rank)[1].rank

				# Update the winners list based on the new highest rank
				winners = [player for player, card in self.played_cards if card.rank == highest_rank]

				# Display the additional played cards
				self.display_played_cards(self.played_cards, winners) 

				# Add the new played cards to the winning cards pile
				self.winning_cards.extend([card for player, card in self.played_cards])

				# Clear the played cards 
				self.played_cards = []
			
		# Winning player receives all the cards.
		winning_player = winners[0]
		print(f"{winning_player.name} wins {len(self.winning_cards)} cards!")
		winning_player.add_cards(self.winning_cards)

		# Clear winning cards pile
		self.winning_cards = []
		return winning_player

	def play_game(self, max_rounds=200):
		round_number = 1
		while any(player.has_cards() for player in self.players) and round_number <= max_rounds:
			print(f"--- Round {round_number} --- \n")
			winner = self.play_round()
			round_number += 1

		if round_number > max_rounds:
			print(f"Game stopped after {max_rounds} rounds.")

		# Declare all remaining players as winners
		print("\nPlayers still in the game:")
		for player in self.players:
			if player.has_cards():
				print(f"{player.name} with {len(player.hand)} cards")



## Interactive Dashboard

In [4]:
class Dashboard:
	def __init__(self):
		self.round_number = 0
		self.players_enabled = False  # Flag to track whether players are enabled

		# Input box for seed
		self.seed_input = widgets.Text(description="Seed:", value="42")

		# Default player names
		default_names = ['Mawkmawk', 'Rap', 'Irun', 'Kurwa', 'Gerbood', 'Comar']

		# Input boxes for player names
		self.player_name_inputs = [widgets.Text(description=f"Player {i+1}:", value=default_names[i]) for i in range(6)]  # Assuming maximum 6 players
        
		# Start Game button
		self.start_game_button = widgets.Button(description="Start Game", button_style="success")
		self.start_game_button.on_click(self.start_game)

		# Output area
		self.output = widgets.Output()

		# Widget layout
		self.widgets = widgets.VBox([
			self.seed_input, 
			widgets.HBox(self.player_name_inputs),
			self.start_game_button, 
			self.output
		])

		# Play buttons
		self.play_rounds_button = widgets.Button(description="Play Round")
		self.play_rounds_button.on_click(lambda _: self.play_rounds(_, 1))
		self.play_10_rounds_button = widgets.Button(description="Play 10 Rounds")
		self.play_10_rounds_button.on_click(lambda _: self.play_rounds(_, 10))
		self.play_100_rounds_button = widgets.Button(description="Play 100 Rounds")
		self.play_100_rounds_button.on_click(lambda _: self.play_rounds(_, 100))
		self.play_1000_rounds_button = widgets.Button(description="Play 1000 Rounds")
		self.play_1000_rounds_button.on_click(lambda _: self.play_rounds(_, 1000))

	def start_game(self, button):
		self.round_number = 0  # Reset round number
		seed = self.seed_input.value.strip()

		# Extract player names from input boxes
		player_names = [input_box.value.strip() for input_box in self.player_name_inputs if input_box.value.strip()]

		# Check if there are at least 2 players
		if len(player_names) < 2:
			print("Please enter names for at least 2 players.")
			return

		# Initialize the Game with the provided seed and player names
		self.game = Game(seed=int(seed), player_names=player_names)  # Initialize the Game with the provided seed
		self.players_enabled = True  # Set players as enabled
		
		# Initialize player buttons with player names
		self.player_buttons = [widgets.Button(description=player.name, disabled=True) for player in self.game.players]  # Set initially disabled
		for button in self.player_buttons:
			button.on_click(self.display_player_hand)

		# Change Start Game button properties
		self.start_game_button.description = "Restart Game"  # Change the button text to "Restart Game"
		self.start_game_button.button_style = "danger"  # Change the button style to red
		self.start_game_button.disabled = False  # Enable the Restart Game button

		# Update the player's stats
		self.update_players()

		# Clear the output
		with self.output:
			clear_output()

		self.widgets.children = [
			self.seed_input,
			self.start_game_button, 
			widgets.HBox(self.player_buttons),
    		widgets.HBox([self.play_rounds_button, self.play_10_rounds_button, self.play_100_rounds_button, self.play_1000_rounds_button]),
			self.output]

	def play_rounds(self, button, num_rounds):
		if not self.players_enabled:
			print("Please start the game first.")
			return
		with self.output:
			for round_number in range(1, num_rounds + 1):
				clear_output()  # Clear the output
				self.round_number += 1
				print(f"--- Round {self.round_number} ---")
				self.game.play_round()
				if num_rounds > 1:
					# Add a delay between rounds for better visualization
					time.sleep(1 / num_rounds)  
				self.update_players()

	def display_player_hand(self, button):
		if not self.players_enabled:
			print("Please start the game first.")
			return
		with self.output:
			clear_output()  # Clear the output
			player_name = button.description.split(' (')[0]  # Extract player's name without the appended number of cards
			player = next((player for player in self.game.players if player.name == player_name), None)
			if player:
				html = '<div style="display: flex; flex-wrap: nowrap;">'
				for card in player.hand:
					html += f'<div style="display: inline-block; margin-right: 5px;">'
					html += f'<img src="{card.image_path}" alt="{card}" style="width: 120px; height: auto;">'
					html += '</div>'
				html += '</div>'
				display(HTML(html))
			else:
				print("Player not found.")
				
	def update_players(self):
		for i, player in enumerate(self.game.players):
			self.player_buttons[i].description = f"{player.name} ({len(player.hand)} cards)"
			self.player_buttons[i].style.disabled = "disabled" if not self.players_enabled else ""  # Gray out the player buttons if game not started
			self.player_buttons[i].disabled = not self.players_enabled  # Disable the player buttons if game not started

dashboard = Dashboard() 
display(dashboard.widgets)